In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [ ]:

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": ""}


dbutils.fs.mount(
source = "abfss://main@mainnasr.dfs.core.windows.net",
mount_point = "/mnt/tokyo_olymp",
extra_configs = configs)

True

In [ ]:
%fs
ls "/mnt/tokyo_olymp"

path,name,size,modificationTime
dbfs:/mnt/tokyo_olymp/transformed-data/,transformed-data/,0,1723042344000


In [ ]:
athletes = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/athletes.csv")
coaches = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/coaches.csv")
entries_gender = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/entriesgender.csv")
teams = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/teams.csv")
medals = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/medals.csv")

In [ ]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [ ]:
entries_gender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [ ]:
# find countries with the highest number of gold medals
top_countries_gold_medals = medals.orderBy('Gold',ascending=False).select("TeamCountry","Gold")
top_countries_gold_medals.show()

+--------------------+----+
|         TeamCountry|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|                Cuba|   7|
|         New Zealand|   7|
|              Brazil|   7|
|              Canada|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|               Kenya|   4|
|              Poland|   4|
|      Czech Republic|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [ ]:
# create temp teams view
teams.createOrReplaceTempView("teams")

In [ ]:
# display the total number of mens events
num_of_events_men=spark.sql("select count(*) AS mens_events from teams WHERE Event = 'Men'")
num_of_events_men.show()

+-----------+
|mens_events|
+-----------+
|        120|
+-----------+



In [ ]:
# display the total number of womens events
num_of_events_women=spark.sql("select count(*) AS womens_events from teams WHERE Event = 'Women'")
num_of_events_women.show()

+-------------+
|womens_events|
+-------------+
|          114|
+-------------+



In [ ]:
from pyspark.sql.functions import count
# count number of discplines in each country
count_disciplines_country = teams.groupBy('Country').agg(count('Discipline').alias('num_disciplines_country')).orderBy('num_disciplines_country',ascending=False)
count_disciplines_country.show()

+--------------------+-----------------------+
|             Country|num_disciplines_country|
+--------------------+-----------------------+
|               Japan|                     48|
|United States of ...|                     47|
|               Italy|                     37|
|             Germany|                     36|
|           Australia|                     35|
|                 ROC|                     34|
|              France|                     33|
|People's Republic...|                     33|
|              Canada|                     30|
|       Great Britain|                     28|
|         Netherlands|                     27|
|              Brazil|                     25|
|              Poland|                     21|
|               Spain|                     20|
|   Republic of Korea|                     19|
|             Hungary|                     14|
|         New Zealand|                     13|
|               Egypt|                     12|
|        Sout

In [ ]:
# display the discipline with the most contestants
most_contestants = entries_gender.orderBy('Total',ascending=False).select("Discipline","Total").limit(1)
most_contestants.show()

+----------+-----+
|Discipline|Total|
+----------+-----+
| Athletics| 2041|
+----------+-----+



In [ ]:
# display the discipline with the least contestants
least_contestants = entries_gender.orderBy('Total',ascending=True).select("Discipline","Total").limit(1)
least_contestants.show()

+--------------------+-----+
|          Discipline|Total|
+--------------------+-----+
|Cycling BMX Frees...|   19|
+--------------------+-----+



In [ ]:
country_most_coaches_df = coaches.dropDuplicates(['Name'])
country_most_coaches_df=country_most_coaches_df.groupBy('Country').agg(count('Name').alias('num_coaches_country')).orderBy('num_coaches_country',ascending=False)
country_most_coaches_df.show()

+--------------------+-------------------+
|             Country|num_coaches_country|
+--------------------+-------------------+
|               Japan|                 35|
|United States of ...|                 28|
|               Spain|                 26|
|           Australia|                 21|
|              Canada|                 15|
|                 ROC|                 12|
|        South Africa|                 12|
|               Italy|                 12|
|People's Republic...|                 11|
|           Venezuela|                 10|
|               Egypt|                 10|
|              France|                 10|
|           Argentina|                 10|
|         Netherlands|                 10|
|             Nigeria|                  9|
|             Germany|                  9|
|         New Zealand|                  8|
|       Great Britain|                  7|
|   Republic of Korea|                  7|
|              Brazil|                  7|
+----------

In [ ]:
country_most_coaches_df.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyo_olymp/transformed-data/countrymostcoaches")

In [ ]:
count_disciplines_country.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyo_olymp/transformed-data/countdisciplinescountry")